# Overview

This Jupyter notebook accompanies the manuscript of Maniaci et al. titled “Concentration and distribution of phytoplankton nitrogen and carbon in the Northwest Atlantic and Indian Ocean: A simple model with applications in satellite remote sensing”. It is designed to illustrate how the data were processed and how the model is fitted to the data.

Please note that this example is designed such that each cell (block of Python code) is run after the previous cell. If you clear or restart, you will need to run through each cell from the start. 

## Data

A full description of the data and how it is processed is provided in the manuscript.

## Model

A full description of the model and how it is tuned to the data is provided in the manuscript.

## Python packages required to run the example
    
To run the example the following packages need to be installed. Run the following cell to load the packages

In [ ]:
#Import packages needed
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.stats import pearsonr
from netCDF4 import Dataset
import matplotlib as mpl
from matplotlib import pyplot as plt 
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## Read data into notebook

In [ ]:
#Read data file in
result = pd.read_csv('Insitu_data.csv')

## Conduct Phytoplankton Carbon and Nitrogen Quantile regression

In [ ]:
#############################################POC###########################################
#Read POC and TChl-a data in
POC           = result.loc[:,'POC'].to_numpy()
TChl_C        = result.loc[:,'turnchl'].to_numpy()
Cruise_id     = result.loc[:,'cruise_id'].to_numpy()
lat_C         = result.loc[:,'lat (deg N)'].to_numpy()
lon_C         = result.loc[:,'long (deg W)'].to_numpy()

#Convert the string 'M' for missing data to a number (-999 chosen)
as_ST         = np.where(POC == 'M')
POC[as_ST]    = -999.
as_ST         = np.where(TChl_C == 'M')
TChl_C[as_ST] = -999.
as_ST         = np.where(lat_C == 'M')
lat_C[as_ST] = -999.
as_ST         = np.where(lon_C == 'M')
lon_C[as_ST] = -999.

#Convert data to float
TChl_C = TChl_C.astype(np.float)
POC    = POC.astype(np.float)
lat_C  = lat_C.astype(np.float)
lon_C  = lon_C.astype(np.float)

#Remove missing data '-999' from both POC and TChl-a arrays (i.e. both arrays need to have data)
as_ST   = np.where(POC > -999)
TChl_C  = TChl_C[as_ST]
POC     = POC[as_ST]
lat_C   = lat_C[as_ST]
lon_C   = lon_C[as_ST]
as_ST   = np.where(TChl_C > -999)
TChl_C  = TChl_C[as_ST]
POC     = POC[as_ST]
lat_C   = lat_C[as_ST]
lon_C   = lon_C[as_ST]
#Print length of data used in the fits
print(len(POC))

#Correlation C and Chl (log10-space)
STATS_REG  = stats.pearsonr(np.log10(TChl_C),np.log10(POC))
print(STATS_REG)

##Log Quantile Regression
##Convert data to a dataframe
data2 = pd.DataFrame({'TChl_C': np.log10(TChl_C), 'POC': np.log10(POC)})

##Run quantile regression at various other levels
mod2  = smf.quantreg('POC ~ TChl_C', data2)
quantiles = np.arange(.01, .99, .01)
def fit_model(q1):
    res2 = mod2.fit(q=q1)
    return [q1, res2.params['Intercept'], res2.params['TChl_C']] + \
            res2.conf_int().loc['Intercept'].tolist() + res2.conf_int().loc['TChl_C'].tolist()
models2 = [fit_model(x) for x in quantiles]
models2 = pd.DataFrame(models2, columns=['q', 'a', 'b', 'la', 'ua', 'lb', 'ub'])
ols2 = smf.ols('POC ~ TChl_C', data2).fit()
ols_ci1 = ols2.conf_int().loc['Intercept'].tolist()
ols_ci2 = ols2.conf_int().loc['TChl_C'].tolist()
ols2 = dict(a = ols2.params['Intercept'],
           b = ols2.params['TChl_C'],
           la = ols_ci1[0],
           ua = ols_ci1[1], 
           lb = ols_ci2[0],
           ub = ols_ci2[1])

##Print summary if necessary
print(models2)

#Sathyendranath parameters
Sathy_slope     = 0.63
Sathy_Intercept = np.log10(64.)

##Select 1% regression
This_study_C_slope_1    = models2.b[0] # 1%
This_study_C_slope_1_SE = models2.b[0]-models2.lb[0] # 1%
This_study_C_Intercept_1    = models2.a[0] # 1%
This_study_C_Intercept_1_SE = models2.a[0]-models2.la[0] # 1%

##Select 50% regression
This_study_C_slope_50    = models2.b[49] # 50%
This_study_C_slope_50_SE = models2.b[49]-models2.lb[49] # 50%
This_study_C_Intercept_50    = models2.a[49] # 50%
This_study_C_Intercept_50_SE = models2.a[49]-models2.la[49] # 50%

res = mod2.fit(q=0.01)
print(res.summary())

# Plot data
XSIZE = 18
YSIZE = 12
#Define the figure window including 3x2 subplots
fig, ([ax1,ax2,ax3],[ax4,ax5,ax6]) = plt.subplots(2,3, figsize=(XSIZE,YSIZE), \
                gridspec_kw={'hspace': 0.25})
fig.patch.set_facecolor('White')
ax1.scatter(data2.TChl_C, data2.POC, alpha=.3, color = 'r')
# Log fit
x2     = np.linspace(data2.TChl_C.min(), data2.TChl_C.max(), 50)
get_y2 = lambda a, b: a + b * x2
for i in range(models2.shape[0]):
    y2 = get_y2(models2.a[i], models2.b[i])
    ax1.plot(x2, y2, linestyle='dotted', color='gray', alpha=.5)
y2 = get_y2(ols2['a'], ols2['b'])
ax1.plot(x2, y2, color='black', label='50% (log10(POC) = 0.48log10(Chl-a) + 2.21)')
#Select 2% regression
y2_TS = get_y2(This_study_C_Intercept_1, This_study_C_slope_1)
ax1.plot(x2, y2_TS, color='blue', label='1% (log10(C) = 0.57log10(Chl-a) + 1.92)')
# Set log scale
legend = ax1.legend()
ax1.set_xlabel('log10(Chl-a)', fontsize=16)
ax1.set_ylabel('log10(POC)', fontsize=16)
ax1.set_title("(A)", fontsize=20) 

ax2.errorbar(models2.q*100., models2.a, yerr=(models2.a-models2.la, models2.ua-models2.a), ms=4, linestyle='None', 
             marker='o', color = 'r', alpha=.5)
ax2.scatter([1,1],[Sathy_Intercept,Sathy_Intercept], label = 'S2009',marker='^', alpha=.5, s = 100, color = 'k')
ax2.plot([0,100],[This_study_C_Intercept_1,This_study_C_Intercept_1], label = 'This Study (1%)',linestyle='-', color = 'b')
ax2.plot([0,100],[This_study_C_Intercept_1-This_study_C_Intercept_1_SE,This_study_C_Intercept_1-This_study_C_Intercept_1_SE], label = 'This Study (1%) lb',linestyle='--', color = 'b')
ax2.plot([0,100],[This_study_C_Intercept_1+This_study_C_Intercept_1_SE,This_study_C_Intercept_1+This_study_C_Intercept_1_SE], label = 'This Study (1%) ub',linestyle='-.', color = 'b')
ax2.plot([0,100],[This_study_C_Intercept_50,This_study_C_Intercept_50], label = 'This Study (50%)',linestyle='-', color = 'k')
ax2.plot([0,100],[This_study_C_Intercept_50-This_study_C_Intercept_50_SE,This_study_C_Intercept_50-This_study_C_Intercept_50_SE], label = 'This Study (50%) lb',linestyle='--', color = 'k')
ax2.plot([0,100],[This_study_C_Intercept_50+This_study_C_Intercept_50_SE,This_study_C_Intercept_50+This_study_C_Intercept_50_SE], label = 'This Study (50%) ub',linestyle='-.', color = 'k')
ax2.set_xlabel('Percentile', fontsize=16)
ax2.set_ylabel('Intercept', fontsize=16)
ax2.set_title("(B)", fontsize=20)
legend = ax2.legend()

ax3.errorbar(models2.q*100., models2.b, yerr=(models2.b-models2.lb, models2.ub-models2.b), ms=4, linestyle='None', 
             marker='o', color = 'r', alpha=.5)
ax3.scatter([1,1],[Sathy_slope,Sathy_slope], label = 'S2009',marker='^', alpha=.5, s = 100, color = 'k')
ax3.plot([0,100],[This_study_C_slope_1,This_study_C_slope_1], label = 'This Study (1%)',linestyle='-', color = 'b')
ax3.plot([0,100],[This_study_C_slope_1-This_study_C_slope_1_SE,This_study_C_slope_1-This_study_C_slope_1_SE], label = 'This Study (1%) lb',linestyle='--', color = 'b')
ax3.plot([0,100],[This_study_C_slope_1+This_study_C_slope_1_SE,This_study_C_slope_1+This_study_C_slope_1_SE], label = 'This Study (1%) ub',linestyle='-.', color = 'b')
ax3.plot([0,100],[This_study_C_slope_50,This_study_C_slope_50], label = 'This Study (50%)',linestyle='-', color = 'k')
ax3.plot([0,100],[This_study_C_slope_50-This_study_C_slope_50_SE,This_study_C_slope_50-This_study_C_slope_50_SE], label = 'This Study (50%) lb',linestyle='--', color = 'k')
ax3.plot([0,100],[This_study_C_slope_50+This_study_C_slope_50_SE,This_study_C_slope_50+This_study_C_slope_50_SE], label = 'This Study (50%) ub',linestyle='-.', color = 'k')
ax3.set_xlabel('Percentile', fontsize=16)
ax3.set_ylabel('Slope', fontsize=16)
ax3.set_title("(C)", fontsize=20)
legend = ax3.legend()

print([10**(This_study_C_Intercept_1), This_study_C_slope_1])

#############################################PON###########################################
#Read PON and TChl-a data in
PON           = result.loc[:,'PON'].to_numpy()
TChl_N        = result.loc[:,'turnchl'].to_numpy()
Cruise_id     = result.loc[:,'cruise_id'].to_numpy()
lat_N         = result.loc[:,'lat (deg N)'].to_numpy()
lon_N         = result.loc[:,'long (deg W)'].to_numpy()

#Convert the string 'M' for missing data to a number (-999 chosen)
as_ST         = np.where(PON == 'M')
PON[as_ST]    = -999.
as_ST         = np.where(TChl_N == 'M')
TChl_N[as_ST] = -999.
as_ST         = np.where(lat_N == 'M')
lat_N[as_ST] = -999.
as_ST         = np.where(lon_N == 'M')
lon_N[as_ST] = -999.

#Convert data to float
TChl_N = TChl_N.astype(np.float)
PON    = PON.astype(np.float)
lat_N  = lat_N.astype(np.float)
lon_N  = lon_N.astype(np.float)

#Remove missing data '-999' from both PON and Tchl arrays (i.e. both arrays need to have data)
as_ST   = np.where(PON > -999)
TChl_N  = TChl_N[as_ST]
PON     = PON[as_ST]
lat_N   = lat_N[as_ST]
lon_N   = lon_N[as_ST]
as_ST   = np.where(TChl_N > -999)
TChl_N  = TChl_N[as_ST]
PON     = PON[as_ST]
lat_N   = lat_N[as_ST]
lon_N   = lon_N[as_ST]
#Print length of data used in the fits
print(len(PON))

#Correlation N and Chl (log10-space)
STATS_REG  = stats.pearsonr(np.log10(TChl_N),np.log10(PON))
print(STATS_REG)

##Log Quantile Regression
##Convert data to a dataframe
data2 = pd.DataFrame({'TChl_N': np.log10(TChl_N), 'PON': np.log10(PON)})

##Run quantile regression at various other levels
mod2  = smf.quantreg('PON ~ TChl_N', data2)
quantiles = np.arange(.01, .99, .01)
def fit_model(q1):
    res2 = mod2.fit(q=q1)
    return [q1, res2.params['Intercept'], res2.params['TChl_N']] + \
            res2.conf_int().loc['Intercept'].tolist() + res2.conf_int().loc['TChl_N'].tolist()
models2 = [fit_model(x) for x in quantiles]
models2 = pd.DataFrame(models2, columns=['q', 'a', 'b', 'la', 'ua', 'lb', 'ub'])
ols2 = smf.ols('PON ~ TChl_N', data2).fit()
ols_ci1 = ols2.conf_int().loc['Intercept'].tolist()
ols_ci2 = ols2.conf_int().loc['TChl_N'].tolist()

ols2 = dict(a = ols2.params['Intercept'],
           b = ols2.params['TChl_N'],
           la = ols_ci1[0],
           ua = ols_ci1[1], 
           lb = ols_ci2[0],
           ub = ols_ci2[1])

##Print summary if necessary
print(models2)

#Select 1% regression
This_study_N_slope_1        = models2.b[0] # 1%
This_study_N_slope_1_SE     = models2.b[0]-models2.lb[0] # 1%
This_study_N_Intercept_1    = models2.a[0] # 1%
This_study_N_Intercept_1_SE = models2.a[0]-models2.la[0] # 1%

##Select 50% regression
This_study_N_slope_50    = models2.b[49] # 50%
This_study_N_slope_50_SE = models2.b[49]-models2.lb[49] # 50%
This_study_N_Intercept_50    = models2.a[49] # 50%
This_study_N_Intercept_50_SE = models2.a[49]-models2.la[49] # 50%

# Plot data
ax4.scatter(data2.TChl_N, data2.PON, alpha=.3, color = 'g')
# Log fit
x2     = np.linspace(data2.TChl_N.min(), data2.TChl_N.max(), 50)
get_y2 = lambda a, b: a + b * x2
for i in range(models2.shape[0]):
    y2 = get_y2(models2.a[i], models2.b[i])
    ax4.plot(x2, y2, linestyle='dotted', color='gray', alpha=.5)
y2 = get_y2(ols2['a'], ols2['b'])
ax4.plot(x2, y2, color='black', label='50% (log10(PON) = 0.53log10(Chl-a) + 1.36)')
#Select 2% regression
y2_TS = get_y2(This_study_N_Intercept_1, This_study_N_slope_1)
ax4.plot(x2, y2_TS, color='blue', label='1% (log10(N) = 0.60log10(Chl-a) + 1.07)')
# Set log scale
legend = ax4.legend()
ax4.set_xlabel('log10(Chl-a)', fontsize=16)
ax4.set_ylabel('log10(PON)', fontsize=16)
ax4.set_title("(D)", fontsize=20)

ax5.errorbar(models2.q*100., models2.a, yerr=(models2.a-models2.la, models2.ua-models2.a), ms=4, linestyle='None', 
             marker='o', color = 'g', alpha=.5)
ax5.plot([0,100],[This_study_N_Intercept_1,This_study_N_Intercept_1], label = 'This Study (1%)',linestyle='-', color = 'b')
ax5.plot([0,100],[This_study_N_Intercept_1-This_study_N_Intercept_1_SE,This_study_N_Intercept_1-This_study_N_Intercept_1_SE], label = 'This Study (1%) lb',linestyle='--', color = 'b')
ax5.plot([0,100],[This_study_N_Intercept_1+This_study_N_Intercept_1_SE,This_study_N_Intercept_1+This_study_N_Intercept_1_SE], label = 'This Study (1%) ub',linestyle='-.', color = 'b')
ax5.plot([0,100],[This_study_N_Intercept_50,This_study_N_Intercept_50], label = 'This Study (50%)',linestyle='-', color = 'k')
ax5.plot([0,100],[This_study_N_Intercept_50-This_study_N_Intercept_50_SE,This_study_N_Intercept_50-This_study_N_Intercept_50_SE], label = 'This Study (50%) lb',linestyle='--', color = 'k')
ax5.plot([0,100],[This_study_N_Intercept_50+This_study_N_Intercept_50_SE,This_study_N_Intercept_50+This_study_N_Intercept_50_SE], label = 'This Study (50%) ub',linestyle='-.', color = 'k')
ax5.set_xlabel('Percentile', fontsize=16)
ax5.set_ylabel('Intercept', fontsize=16)
ax5.set_title("(E)", fontsize=20)
legend = ax5.legend()

ax6.errorbar(models2.q*100., models2.b, yerr=(models2.b-models2.lb, models2.ub-models2.b), ms=4, linestyle='None', 
             marker='o', color = 'g', alpha=.5)
ax6.plot([0,100],[This_study_N_slope_1,This_study_N_slope_1], label = 'This Study (1%)',linestyle='-', color = 'b')
ax6.plot([0,100],[This_study_N_slope_1-This_study_N_slope_1_SE,This_study_N_slope_1-This_study_N_slope_1_SE], label = 'This Study (1%) lb',linestyle='--', color = 'b')
ax6.plot([0,100],[This_study_N_slope_1+This_study_N_slope_1_SE,This_study_N_slope_1+This_study_N_slope_1_SE], label = 'This Study (1%) ub',linestyle='-.', color = 'b')
ax6.plot([0,100],[This_study_N_slope_50,This_study_N_slope_50], label = 'This Study (50%)',linestyle='-', color = 'k')
ax6.plot([0,100],[This_study_N_slope_50-This_study_N_slope_50_SE,This_study_N_slope_50-This_study_N_slope_50_SE], label = 'This Study (50%) lb',linestyle='--', color = 'k')
ax6.plot([0,100],[This_study_N_slope_50+This_study_N_slope_50_SE,This_study_N_slope_50+This_study_N_slope_50_SE], label = 'This Study (50%) ub',linestyle='-.', color = 'k')
ax6.set_xlabel('Percentile', fontsize=16)
ax6.set_ylabel('Slope', fontsize=16)
ax6.set_title("(F)", fontsize=20)
legend = ax6.legend()

plt.show()
print([10**(This_study_N_Intercept_1), This_study_N_slope_1])


## Ratio of Carbon to Nitrogen as a function of Chl-a

In [ ]:
#Read POC and TChl-a data in
POC_1           = result.loc[:,'POC'].to_numpy()
PON_1           = result.loc[:,'PON'].to_numpy()
TChl_NC         = result.loc[:,'turnchl'].to_numpy()
Cruise_id     = result.loc[:,'cruise_id'].to_numpy()

#Convert the string 'M' for missing data to a number (-999 chosen)
as_ST         = np.where(POC_1 == 'M')
POC_1[as_ST]  = -999.
as_ST         = np.where(PON_1 == 'M')
PON_1[as_ST]  = -999.
as_ST         = np.where(TChl_NC == 'M')
TChl_NC[as_ST]= -999.

#Convert data to float
TChl_NC = TChl_NC.astype(np.float)
POC_1   = POC_1.astype(np.float)
PON_1   = PON_1.astype(np.float)

#Remove missing data '-999' from both arrays (i.e. both arrays need to have data)
as_ST   = np.where(POC_1 > -999)
TChl_NC = TChl_NC[as_ST]
POC_1   = POC_1[as_ST]
PON_1   = PON_1[as_ST]
as_ST   = np.where(PON_1 > -999)
TChl_NC = TChl_NC[as_ST]
POC_1   = POC_1[as_ST]
PON_1   = PON_1[as_ST]
as_ST   = np.where(TChl_NC > -999)
TChl_NC = TChl_NC[as_ST]
POC_1   = POC_1[as_ST]
PON_1   = PON_1[as_ST]

# Ratio
Ratio    = POC_1/PON_1
CHL_SIM2 = np.log10(np.linspace(0.01, 100, 1000))

print(np.mean(Ratio))
print(np.min(Ratio))
print(np.max(Ratio))
print(np.std(Ratio))

# Ensemble of uncertainty in ratio
Ratio_matrix = np.empty([len(CHL_SIM2),81])
get_y2 = lambda a, b: a + b * CHL_SIM2
count_i = 0
for j in range(3):
    if j == 0:
        C_SL = This_study_C_slope_1
    if j == 1:
        C_SL = This_study_C_slope_1+This_study_C_slope_1_SE
    if j == 2:
        C_SL = This_study_C_slope_1-This_study_C_slope_1_SE
    for i in range(3):
        if i == 0:
            C_IN = This_study_C_Intercept_1
        if i == 1:
            C_IN = This_study_C_Intercept_1+This_study_C_Intercept_1_SE
        if i == 2:
            C_IN = This_study_C_Intercept_1-This_study_C_Intercept_1_SE
        for k in range(3):
            if k == 0:
                N_SL = This_study_N_slope_1
            if k == 1:
                N_SL = This_study_N_slope_1+This_study_N_slope_1_SE
            if k == 2:
                N_SL = This_study_N_slope_1-This_study_N_slope_1_SE
            for l in range(3):
                if l == 0:
                    N_IN = This_study_N_Intercept_1
                if l == 1:
                    N_IN = This_study_N_Intercept_1+This_study_N_Intercept_1_SE
                if l == 2:
                    N_IN = This_study_N_Intercept_1-This_study_N_Intercept_1_SE      
                
                Phyto_C_L0 = 10**(get_y2(C_IN, C_SL))
                Phyto_N_L0 = 10**(get_y2(N_IN, N_SL))
                Ratio_matrix[:,count_i] = Phyto_C_L0/Phyto_N_L0
                count_i = count_i + 1
# Extract min and max of Ensemble as uncertainty
Ratio_low  = CHL_SIM2*0  
Ratio_high = CHL_SIM2*0  
for j in range(len(CHL_SIM2)): 
    Temp_Rat = Ratio_matrix[j,:]
    Ratio_low[j]  = np.min(Temp_Rat)
    Ratio_high[j] = np.max(Temp_Rat)

#Regression and compute stats
Phyto_C_L = get_y2(This_study_C_Intercept_1, This_study_C_slope_1)
Phyto_N_L = get_y2(This_study_N_Intercept_1, This_study_N_slope_1)
Phyto_C_L = 10**(Phyto_C_L)
Phyto_N_L = 10**(Phyto_N_L)
Ratio_mod_L = Phyto_C_L / Phyto_N_L
get_y21       = lambda a, b: a + b * np.log10(TChl_NC)
Phyto_C_L0_1 = 10**(get_y21(This_study_C_Intercept_1, This_study_C_slope_1))
Phyto_N_L0_1 = 10**(get_y21(This_study_N_Intercept_1, This_study_N_slope_1))
Ratio_matrix_1 = Phyto_C_L0_1/Phyto_N_L0_1
print('Start phyto C/N stats')
print(np.mean(Ratio_matrix_1))
print(np.std(Ratio_matrix_1))
print(np.min(Ratio_matrix_1))
print(np.max(Ratio_matrix_1))
print('Start phyto C/Chl stats')
Ratio_matrix_1 = Phyto_C_L0_1/TChl_NC
print(np.mean(Ratio_matrix_1))
print(np.std(Ratio_matrix_1))
print(np.min(Ratio_matrix_1))
print(np.max(Ratio_matrix_1))
print('Start phyto N/Chl stats')
Ratio_matrix_1 = Phyto_N_L0_1/TChl_NC
print(np.mean(Ratio_matrix_1))
print(np.std(Ratio_matrix_1))
print(np.min(Ratio_matrix_1))
print(np.max(Ratio_matrix_1))


# Plot data
XSIZE = 12
YSIZE = 10
#Define the figure window including 2 subplots orientated horizontally
fig, (ax1) = plt.subplots(1,1, figsize=(XSIZE,YSIZE), \
                gridspec_kw={'hspace': 0})
fig.patch.set_facecolor('White')
ax2 = ax1.twinx()
#Log
ax1.scatter(TChl_NC, Ratio, alpha=.1, color = 'k', s = 100)
# naming the x axis
ax1.set_xlabel('Chl-a (mg m$^{−3}$)', fontsize=30)
# naming the y axis
ax1.set_ylabel('POC/PON', fontsize=30)
ax1.tick_params(axis='x', labelsize= 15)
ax1.tick_params(axis='y', labelsize= 15)
ax1.set_ylim(3, 17);
# Plot title
#ax1.set_title("Ratio", fontsize=17)
# Set log scale)
ax1.set_xscale('log')
ax1.plot([0.01,100],[6.625,6.625], color='r', label = 'Redfield', linewidth=3.0, linestyle=':')
ax1.plot(10**(CHL_SIM2),Ratio_mod_L, color='b', label = 'Phyto C/N', linewidth=3.0)
ax1.plot(10**(CHL_SIM2),Ratio_low, color='b', label = 'Phyto C/N (lb)',linestyle='-.', linewidth=3.0)
ax1.plot(10**(CHL_SIM2),Ratio_high, color='b', label = 'Phyto C/N (ub)',linestyle='--', linewidth=3.0)
ax2.set_ylabel('C/N', color='b', fontsize=30)
ax2.tick_params(axis='x', labelcolor='b',labelsize= 15)
ax2.tick_params(axis='y', labelcolor='b',labelsize= 15)
ax2.set_ylim(3, 17);
legend = ax1.legend(fontsize=20)
plt.show()

## HPLC analysis of pigments and phytoplankrton group C:N

In [ ]:
#Read all pigment taxa used in Sath-et-al-2009
HPLC_alloxanthin           = result.loc[:,'allox'].to_numpy()
HPLC_but                   = result.loc[:,'but19'].to_numpy()
HPLC_chl_b                 = result.loc[:,'chlb'].to_numpy()
HPLC_Chlc_1_2              = result.loc[:,'chlc12'].to_numpy()
HPLC_chlc_3                = result.loc[:,'chlc3'].to_numpy()
HPLC_diadinoxanthin        = result.loc[:,'diadinox'].to_numpy()
HPLC_fucoxanthin           = result.loc[:,'fucox'].to_numpy()
HPLC_hex                   = result.loc[:,'hex19'].to_numpy()
HPLC_chla                  = result.loc[:,'hplchla'].to_numpy()
HPLC_divinyl_chla          = result.loc[:,'pctDVa'].to_numpy()
HPLC_divinyl_chlb          = result.loc[:,'pctDVb'].to_numpy()
HPLC_peridinin             = result.loc[:,'perid'].to_numpy()
HPLC_zeaxanthin            = result.loc[:,'zea'].to_numpy()

#Convert the sting 'M' to -999
as_ST         = np.where(HPLC_alloxanthin == 'M')
HPLC_alloxanthin[as_ST]= -999.
as_ST         = np.where(HPLC_but == 'M')
HPLC_but[as_ST]= -999.
as_ST         = np.where(HPLC_chl_b == 'M')
HPLC_chl_b[as_ST]= -999.
as_ST         = np.where(HPLC_Chlc_1_2 == 'M')
HPLC_Chlc_1_2[as_ST]= -999.
as_ST         = np.where(HPLC_chlc_3 == 'M')
HPLC_chlc_3[as_ST]= -999.
as_ST         = np.where(HPLC_diadinoxanthin == 'M')
HPLC_diadinoxanthin[as_ST]= -999.
as_ST         = np.where(HPLC_fucoxanthin == 'M')
HPLC_fucoxanthin[as_ST]= -999.
as_ST         = np.where(HPLC_hex == 'M')
HPLC_hex[as_ST]= -999.
as_ST         = np.where(HPLC_chla == 'M')
HPLC_chla[as_ST]= -999.
as_ST         = np.where(HPLC_divinyl_chla == 'M')
HPLC_divinyl_chla[as_ST]= -999.
as_ST         = np.where(HPLC_divinyl_chlb == 'M')
HPLC_divinyl_chlb[as_ST]= -999.
as_ST         = np.where(HPLC_peridinin == 'M')
HPLC_peridinin[as_ST]= -999.
as_ST         = np.where(HPLC_zeaxanthin == 'M')
HPLC_zeaxanthin[as_ST]= -999.

#Convert data to float
HPLC_alloxanthin = HPLC_alloxanthin.astype(np.float)
HPLC_but   = HPLC_but.astype(np.float)
HPLC_chl_b   = HPLC_chl_b.astype(np.float)
HPLC_Chlc_1_2   = HPLC_Chlc_1_2.astype(np.float)
HPLC_chlc_3   = HPLC_chlc_3.astype(np.float)
HPLC_diadinoxanthin   = HPLC_diadinoxanthin.astype(np.float)
HPLC_fucoxanthin   = HPLC_fucoxanthin.astype(np.float)
HPLC_hex   = HPLC_hex.astype(np.float)
HPLC_chla   = HPLC_chla.astype(np.float)
HPLC_divinyl_chla   = HPLC_divinyl_chla.astype(np.float)
HPLC_divinyl_chlb   = HPLC_divinyl_chlb.astype(np.float)
HPLC_peridinin   = HPLC_peridinin.astype(np.float)
HPLC_zeaxanthin   = HPLC_zeaxanthin.astype(np.float)


#Remove missing data '-999' from both arrays
for j in range(13):
    if j == 0:
        as_ST = np.where(HPLC_alloxanthin > -999)
    if j == 1:
        as_ST = np.where(HPLC_but > -999)
    if j == 2:
        as_ST = np.where(HPLC_chl_b > -999)
    if j == 3:
        as_ST = np.where(HPLC_Chlc_1_2 > -999)
    if j == 4:
        as_ST = np.where(HPLC_chlc_3 > -999)
    if j == 5:
        as_ST = np.where(HPLC_diadinoxanthin > -999)
    if j == 6:
        as_ST = np.where(HPLC_fucoxanthin > -999)
    if j == 7:
        as_ST = np.where(HPLC_hex > -999)
    if j == 8:
        as_ST = np.where(HPLC_chla > -999)
    if j == 9:
        as_ST = np.where(HPLC_divinyl_chla > -999)
    if j == 10:
        as_ST = np.where(HPLC_divinyl_chlb > -999)
    if j == 11:
        as_ST = np.where(HPLC_peridinin > -999)
    if j == 12:
        as_ST = np.where(HPLC_zeaxanthin > -999)
    HPLC_alloxanthin = HPLC_alloxanthin[as_ST]
    HPLC_but = HPLC_but[as_ST]
    HPLC_chl_b = HPLC_chl_b[as_ST]
    HPLC_Chlc_1_2 = HPLC_Chlc_1_2[as_ST]
    HPLC_chlc_3 = HPLC_chlc_3[as_ST]
    HPLC_diadinoxanthin = HPLC_diadinoxanthin[as_ST]
    HPLC_fucoxanthin = HPLC_fucoxanthin[as_ST]
    HPLC_hex = HPLC_hex[as_ST]
    HPLC_chla = HPLC_chla[as_ST]
    HPLC_divinyl_chla = HPLC_divinyl_chla[as_ST]
    HPLC_divinyl_chlb = HPLC_divinyl_chlb[as_ST]
    HPLC_peridinin = HPLC_peridinin[as_ST]
    HPLC_zeaxanthin = HPLC_zeaxanthin[as_ST]

# Compute Ratios
Ratio_HPLC_alloxanthin = HPLC_alloxanthin/HPLC_chla
Ratio_HPLC_but = HPLC_but/HPLC_chla
Ratio_HPLC_chl_b = HPLC_chl_b/HPLC_chla
Ratio_HPLC_Chlc_1_2 = HPLC_Chlc_1_2/HPLC_chla
Ratio_HPLC_chlc_3 = HPLC_chlc_3/HPLC_chla
Ratio_HPLC_diadinoxanthin = HPLC_diadinoxanthin/HPLC_chla
Ratio_HPLC_fucoxanthin = HPLC_fucoxanthin/HPLC_chla
Ratio_HPLC_hex = HPLC_hex/HPLC_chla
Ratio_HPLC_divinyl_chla = HPLC_divinyl_chla/HPLC_chla
Ratio_HPLC_divinyl_chlb = HPLC_divinyl_chlb/HPLC_chla
Ratio_HPLC_peridinin = HPLC_peridinin/HPLC_chla
Ratio_HPLC_zeaxanthin = HPLC_zeaxanthin/HPLC_chla

# Implement Sathyendranath criteria
INDEX_Phyto_group = HPLC_chla*0.
# 1 = Prymnesiophytes
for j in range(len(HPLC_chla)):  
    # presymiophytes
    if Ratio_HPLC_chlc_3[j] > 0.035:
        if Ratio_HPLC_divinyl_chla[j] < 0.1:
            if Ratio_HPLC_divinyl_chlb[j] < 0.1:
                if Ratio_HPLC_zeaxanthin[j] < 0.01:
                    if Ratio_HPLC_peridinin[j] < 0.1:
                        if Ratio_HPLC_chl_b[j] < 0.1:
                            if Ratio_HPLC_hex[j] > 0.05:
                                if Ratio_HPLC_but[j] > 0.05:
                                    INDEX_Phyto_group[j] = 1
    # prochlorococcus
    if Ratio_HPLC_divinyl_chla[j] > 0.5:
        if Ratio_HPLC_divinyl_chlb[j] > 0.5:
            if Ratio_HPLC_fucoxanthin[j] < 0.01:
                if Ratio_HPLC_chlc_3[j] < 0.01:
                    if Ratio_HPLC_hex[j] < 0.2:
                        INDEX_Phyto_group[j] = 2
    # diatoms
    if Ratio_HPLC_fucoxanthin[j] > 0.4:
        if Ratio_HPLC_Chlc_1_2[j] > 0.1:
            if Ratio_HPLC_chlc_3[j] < 0.01:
                if Ratio_HPLC_zeaxanthin[j] < 0.01:
                    if Ratio_HPLC_hex[j] < 0.1:
                        if Ratio_HPLC_chl_b[j] < 0.1:
                            if Ratio_HPLC_diadinoxanthin[j] > 0.01:
                                INDEX_Phyto_group[j] = 3
    # cyanobacteria
    if Ratio_HPLC_zeaxanthin[j] > 0.1:
        if Ratio_HPLC_divinyl_chla[j] < 0.2:
            if Ratio_HPLC_fucoxanthin[j] < 0.1:
                if Ratio_HPLC_chl_b[j] < 0.2:
                    if Ratio_HPLC_peridinin[j] < 0.03:
                        if Ratio_HPLC_hex[j] < 0.2:
                            if Ratio_HPLC_chlc_3[j] < 0.035:
                                INDEX_Phyto_group[j] = 4
    # green
    if Ratio_HPLC_chl_b[j] > 0.1:
        if Ratio_HPLC_divinyl_chla[j] > 0.1:
            if Ratio_HPLC_divinyl_chlb[j] < 0.1:
                if Ratio_HPLC_fucoxanthin[j] < 0.01:
                    if Ratio_HPLC_Chlc_1_2[j] < 0.1:
                        if Ratio_HPLC_hex[j] < 0.2:
                            if Ratio_HPLC_alloxanthin[j] < 0.05:
                                INDEX_Phyto_group[j] = 5
    # dianoflagellates
    if Ratio_HPLC_fucoxanthin[j] > 0.25:
        if Ratio_HPLC_peridinin[j] > 0.4:
            if Ratio_HPLC_hex[j] < 0.2:
                if Ratio_HPLC_chl_b[j] < 0.1:
                    INDEX_Phyto_group[j] = 6
    
#log-10 transform for model
HPLC_chla = np.log10(HPLC_chla)

# presymiophytes
asd = np.where(INDEX_Phyto_group == 1)
HPLC_CHL_Prymes = HPLC_chla[asd]
get_y2 = lambda a, b: a + b * HPLC_CHL_Prymes
PON_Prymes = 10**(get_y2(This_study_N_Intercept_1, This_study_N_slope_1))
POC_Prymes = 10**(get_y2(This_study_C_Intercept_1, This_study_C_slope_1))
POC_PON_Prymes = POC_Prymes / PON_Prymes

print(len(INDEX_Phyto_group[asd]))
print(np.mean(PON_Prymes))
print(np.mean(POC_Prymes))
print(np.mean(POC_PON_Prymes))
print(np.min(POC_PON_Prymes))
print(np.max(POC_PON_Prymes))

# procolococcus
asd = np.where(INDEX_Phyto_group == 2)
HPLC_CHL_Pro = HPLC_chla[asd]
get_y2 = lambda a, b: a + b * HPLC_CHL_Pro
PON_Pro = 10**(get_y2(This_study_N_Intercept_1, This_study_N_slope_1))
POC_Pro = 10**(get_y2(This_study_C_Intercept_1, This_study_C_slope_1))
POC_PON_Pro = POC_Pro / PON_Pro

print(len(INDEX_Phyto_group[asd]))
print(np.mean(PON_Pro))
print(np.mean(POC_Pro))
print(np.mean(POC_PON_Pro))
print(np.min(POC_PON_Pro))
print(np.max(POC_PON_Pro))

# diatoms
asd = np.where(INDEX_Phyto_group == 3)
HPLC_CHL_dia = HPLC_chla[asd]
get_y2 = lambda a, b: a + b * HPLC_CHL_dia
PON_dia = 10**(get_y2(This_study_N_Intercept_1, This_study_N_slope_1))
POC_dia = 10**(get_y2(This_study_C_Intercept_1, This_study_C_slope_1))
POC_PON_dia = POC_dia / PON_dia

print(len(INDEX_Phyto_group[asd]))
print(np.mean(PON_dia))
print(np.mean(POC_dia))
print(np.mean(POC_PON_dia))
print(np.min(POC_PON_dia))
print(np.max(POC_PON_dia))


# cyanobacteria
asd = np.where(INDEX_Phyto_group == 4)
HPLC_CHL_cya = HPLC_chla[asd]
get_y2 = lambda a, b: a + b * HPLC_CHL_cya
PON_cya = 10**(get_y2(This_study_N_Intercept_1, This_study_N_slope_1))
POC_cya = 10**(get_y2(This_study_C_Intercept_1, This_study_C_slope_1))
POC_PON_cya = POC_cya / PON_cya

print(len(INDEX_Phyto_group[asd]))
print(np.mean(PON_cya))
print(np.mean(POC_cya))
print(np.mean(POC_PON_cya))
print(np.min(POC_PON_cya))
print(np.max(POC_PON_cya))

# green algae
asd = np.where(INDEX_Phyto_group == 5)
HPLC_CHL_green = HPLC_chla[asd]
get_y2 = lambda a, b: a + b * HPLC_CHL_green
PON_green = 10**(get_y2(This_study_N_Intercept_1, This_study_N_slope_1))
POC_green = 10**(get_y2(This_study_C_Intercept_1, This_study_C_slope_1))
POC_PON_green = POC_green / PON_green

print(len(INDEX_Phyto_group[asd]))
print(np.mean(PON_green))
print(np.mean(POC_green))
print(np.mean(POC_PON_green))
print(np.min(POC_PON_green))
print(np.max(POC_PON_green))

# dinoflagellates
asd = np.where(INDEX_Phyto_group == 6)
HPLC_CHL_dino = HPLC_chla[asd]
get_y2 = lambda a, b: a + b * HPLC_CHL_dino
PON_dino = 10**(get_y2(This_study_N_Intercept_1, This_study_N_slope_1))
POC_dino = 10**(get_y2(This_study_C_Intercept_1, This_study_C_slope_1))
POC_PON_dino = POC_dino / PON_dino

print(len(INDEX_Phyto_group[asd]))
print(np.mean(PON_dino))
print(np.mean(POC_dino))
print(np.mean(POC_PON_dino))


# Plot data
XSIZE = 20
YSIZE = 12
#Define the figure window including 2 subplots orientated horizontally
fig, (ax1) = plt.subplots(1,1, figsize=(XSIZE,YSIZE), \
                gridspec_kw={'hspace': 0})
fig.patch.set_facecolor('White')
ax1.boxplot(POC_PON_Prymes, positions = [0.1], whiskerprops = dict(linewidth=2), boxprops = dict(linewidth=2), medianprops = dict(linewidth=2), capprops= dict(linewidth=2))
ax1.boxplot(POC_PON_Pro, positions = [0.3], whiskerprops = dict(linewidth=2), boxprops = dict(linewidth=2), medianprops = dict(linewidth=2), capprops= dict(linewidth=2))
ax1.boxplot(POC_PON_dia, positions = [0.50], whiskerprops = dict(linewidth=2), boxprops = dict(linewidth=2), medianprops = dict(linewidth=2), capprops= dict(linewidth=2))
ax1.boxplot(POC_PON_cya, positions = [0.7], whiskerprops = dict(linewidth=2), boxprops = dict(linewidth=2), medianprops = dict(linewidth=2), capprops= dict(linewidth=2))
ax1.boxplot(POC_PON_green, positions = [0.90], whiskerprops = dict(linewidth=2), boxprops = dict(linewidth=2), medianprops = dict(linewidth=2), capprops= dict(linewidth=2))
ax1.plot([0.0,1],[6.625,6.625], color='red', label = 'Redfield', linewidth=3.0, linestyle=':')
ax1.set_ylim([4,10]) 
ax1.set_xlim([0,1])
ax1.set_xticklabels(['Prymnesiophytes','Prochlorococcus','Diatoms','Cyanobacteria','Green algae'], fontsize=40)
ax1.set_ylabel('C/N', fontsize=30)
ax1.tick_params(axis='x', labelsize= 25)
ax1.tick_params(axis='y', labelsize= 25)
legend = ax1.legend(fontsize=20)
plt.show()

## Map of data points

In [ ]:
#NW Atlantic
Max_lon    = -35  
Min_lon    = -75
Max_lat    = 70
Min_lat    = 30

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,16), \
                gridspec_kw={'hspace': 0.05}, subplot_kw={'projection': ccrs.PlateCarree()})
fig.patch.set_facecolor('White')
ax1.set_title('NW Atlantic', fontsize = 20) 
ax1.coastlines()
ax1.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='k', facecolor='darkgray'))
ax1.set_xlim([Min_lon, Max_lon])
ax1.set_ylim([Min_lat, Max_lat])
ax1.scatter(lon_C*(-1), lat_C,  transform=ccrs.PlateCarree(), \
            marker = 'o', color = 'r',s = 100, zorder=11, label = 'Data') 
ax1.scatter(lon_N*(-1), lat_N,  transform=ccrs.PlateCarree(), \
            marker = 'o', color = 'r',s = 100, zorder=11) 
gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.bottom_labels = True
gl.right_labels = False
gl.xlines = True
gl.ylabel_style = {'fontsize': 15, 'color': 'k'}
gl.xlabel_style = {'fontsize': 15, 'color': 'k'}
#Import array
ax1.legend(loc="upper right", fontsize = 20)

#Arabian Sea
Max_lon    = 75  
Min_lon    = 35
Max_lat    = 35
Min_lat    = -5

ax2.set_title('Arabian Sea', fontsize = 20) 
ax2.coastlines()
ax2.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='k', facecolor='darkgray'))
ax2.set_xlim([Min_lon, Max_lon])
ax2.set_ylim([Min_lat, Max_lat])
ax2.scatter(lon_C*(-1), lat_C,  transform=ccrs.PlateCarree(), \
            marker = 'o', color = 'r',s = 100, zorder=11, label = 'Data') 
ax2.scatter(lon_N*(-1), lat_N,  transform=ccrs.PlateCarree(), \
            marker = 'o', color = 'r',s = 100, zorder=11) 
gl = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.bottom_labels = True
gl.right_labels = False
gl.xlines = True
gl.ylabel_style = {'fontsize': 15, 'color': 'k'}
gl.xlabel_style = {'fontsize': 15, 'color': 'k'}

plt.show() 